In [1]:
!pip install -q wandb
import wandb
wandb.login()
# api key 3a629afb6d101b0cc3a0123089694d3b03f196e3

     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 145 kB 66.1 MB/s 
     |████████████████████████████████| 181 kB 73.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [16]:
config = {
    "dataset": "MNIST",
    "gpu": "colab",
    "model": "Autoencoder",
    "learning_rate": 0.005,
    "batch_size": 128,
}
wandb.init(project="week11_wandb_example", config=config)
wandb.run.name = "wandb example 2"

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [4]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [6]:
import torchvision
import torchvision.transforms as transforms

train_dataset = torchvision.datasets.MNIST(root="MNIST_data/",
                                train=True,
                                transform = transforms.ToTensor(),
                                download=True)

test_dataset = torchvision.datasets.MNIST(root="MNIST_data/",
                                train = False,
                                transform = transforms.ToTensor(),
                                download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [7]:
batch_size = 128

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [17]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.linear = nn.Linear(784, 256)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(256, 784)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        return x

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return z, x_hat

In [18]:
model = AutoEncoder().to(device)

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [20]:
criterion = nn.MSELoss()

In [21]:
samples = []
for i in range(10):
  samples.append(test_dataset[i][0].view(-1, 784).to(device))
samples = torch.stack(samples).to(device)
samples.shape

torch.Size([10, 1, 784])

In [22]:
epochs = 31

model.train()
for epoch in range(epochs):
    model.train()
    avg_cost = 0
    total_batch_num = len(train_dataloader)

    for step, batch in enumerate(train_dataloader):
        b_x, b_y = batch
        b_x = b_x.view(-1, 784).to(device)
        z, b_x_hat = model.forward(b_x)
        loss = criterion(b_x_hat, b_x)

        avg_cost += loss / total_batch_num
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({"loss": loss})
    
        # observe differences
        if step % 5 == 0:
            model.eval()
            with torch.no_grad():
                test_z, test_output = model.forward(samples)
            test_output = test_output.detach().cpu().reshape(10,28,28)
            wandb.log({'testset x hat': [wandb.Image(i) for i in test_output]})
    print(f'Epoch: {epoch + 1} / {epochs}, cost : {avg_cost}')

Epoch: 1 / 31, cost : 0.023875270038843155
Epoch: 2 / 31, cost : 0.005723295733332634
Epoch: 3 / 31, cost : 0.003554689232259989
Epoch: 4 / 31, cost : 0.0027080769650638103
Epoch: 5 / 31, cost : 0.0022437842562794685
Epoch: 6 / 31, cost : 0.0019441739423200488
Epoch: 7 / 31, cost : 0.001738815102726221
Epoch: 8 / 31, cost : 0.0015811019111424685
Epoch: 9 / 31, cost : 0.0014726631343364716
Epoch: 10 / 31, cost : 0.0013753339881077409
Epoch: 11 / 31, cost : 0.0013091786531731486
Epoch: 12 / 31, cost : 0.0012518049916252494
Epoch: 13 / 31, cost : 0.0011967950267717242
Epoch: 14 / 31, cost : 0.0011609889334067702
Epoch: 15 / 31, cost : 0.0011194038670510054
Epoch: 16 / 31, cost : 0.0010990476002916694
Epoch: 17 / 31, cost : 0.0010756117990240455
Epoch: 18 / 31, cost : 0.0010542682139202952
Epoch: 19 / 31, cost : 0.00103523931466043
Epoch: 20 / 31, cost : 0.0010108628775924444
Epoch: 21 / 31, cost : 0.0010010695550590754
Epoch: 22 / 31, cost : 0.0009867483749985695
Epoch: 23 / 31, cost : 0.